In [1]:
from scapy.all import *

In [3]:
f = rdpcap('../samples/download.pcap')

In [7]:
# Below defines rough implementation

class CoreStructure:
    core_structure = dict()
    raw_scapy_packets = None
    
    def __init__(self, scapy_packets):
        # We are assuming scapy_packets is a perfectly valid scapy packet structure
        self.raw_scapy_packets = scapy_packets
        
    def start(self):
        for packet in self.raw_scapy_packets[:50]:
            l3 = ''
            l4 = ''
            s_ip_addr = None
            d_ip_addr = None
            s_port = None
            d_port = None
            if packet.getlayer(1).name == 'IP':
                l3 = 'IPv4'
                s_ip_addr = packet.getlayer(1).src
                d_ip_addr = packet.getlayer(1).dst
                if packet.getlayer(2).name and packet.getlayer(2).name == 'TCP':
                    l4_packet = packet.getlayer(2) # Storing reference
                    l4 = l4_packet.name
                    s_port = getattr(l4_packet, 'sport')
                    d_port = getattr(l4_packet, 'dport')
                    s_socket = f'{s_ip_addr}:{s_port}'
                    d_socket = f'{d_ip_addr}:{d_port}'
                    # Check if the particular entry exists
                    if not (s_socket, d_socket) in self.core_structure:
                        # Remember src dest will be reversed in packet but
                        # the actual connection is still the same
                        # So we are checking if the reverse tuple exists
                        if not (d_socket, s_socket) in self.core_structure:
                            # If not create a new
                            self.core_structure[(s_socket, d_socket)] = {
                                "l2_proto": packet.name,
                                "l3_proto": l3,
                                "l4_proto": l4_packet.name,
                                "seq": getattr(l4_packet, 'seq'),
                                "up_data": 0,
                                "down_data": 0
                            }
                        else:
                            # This is a reverse scenario, so we will have to swap the src dest
                            # THIS IS A TEMPORARY FIX
                            s_port, d_port = d_port, s_port
                            s_ip_addr, d_ip_addr = d_ip_addr, s_ip_addr
                            s_socket, d_socket = d_socket, s_socket
                    else:
                        current_entry = self.core_structure[(s_socket, d_socket)] # Storing reference
                        print(current_entry)
                        # I am not sure if the below statement will produce correct results
                        # So for now this exists and "kinda" works but shouldn't be assumed final
                        current_entry["up_data"] += getattr(l4_packet, 'seq') - current_entry["seq"]
                        # Update
                        current_entry["seq"] = getattr(l4_packet, 'seq')

In [8]:
core_struct = CoreStructure(f)

In [9]:
core_struct.start()

{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 3495802807, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 3495802808, 'up_data': 1, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 442586608, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 3495802808, 'up_data': 1, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 1338928663, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 4045689038, 'up_data': 0, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 1338928664, 'up_data': 1, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 4045689039, 'up_data': 1, 'down_data': 0}
{'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 1338928664, 'up_data': 1, 

In [10]:
print(core_struct.core_structure)

{('192.168.0.104:36026', '172.67.216.140:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 2540327016, 'up_data': 0, 'down_data': 0}, ('192.168.0.104:54960', '104.17.32.82:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 740875834, 'up_data': 0, 'down_data': 0}, ('192.168.0.104:60028', '185.199.110.153:80'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 3495803229, 'up_data': 422, 'down_data': 0}, ('192.168.0.104:60030', '185.199.110.153:80'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 442586609, 'up_data': 1, 'down_data': 0}, ('192.168.0.104:34690', '185.199.108.153:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 1338929181, 'up_data': 518, 'down_data': 0}, ('192.168.0.104:34692', '185.199.108.153:443'): {'l2_proto': 'Ethernet', 'l3_proto': 'IPv4', 'l4_proto': 'TCP', 'seq': 4045689556, 'up_data': 518, 'down_data': 0}}


## Good enough so far 😄

In [7]:
f = rdpcap('../samples/download_tcp_problem.pcap')

In [8]:
for packet in f:
    print(getattr(packet, 'flags'), end=' ')

DF DF DF DF DF DF DF DF DF DF DF DF DF DF DF DF DF 

In [9]:
for packet in f:
    print(packet['TCP'].flags)

S
SA
A
PA
A
A
A
A
A
PA
A
PA
FA
PA
R
FA
R


## DNS Stuff

In [12]:
f = rdpcap('../samples/dns_queries.pcap')

In [28]:
for packet in f:
    # print(float(packet.time))
    dns_packet = packet['DNS']
    dns_query = getattr(dns_packet, 'qd')
    print(getattr(dns_query, 'qtype'))
    print(dns_packet.fields)

16
{'length': None, 'id': 4146, 'qr': 0, 'opcode': 0, 'aa': 0, 'tc': 0, 'rd': 1, 'ra': 0, 'z': 0, 'ad': 0, 'cd': 0, 'rcode': 0, 'qdcount': 1, 'ancount': 0, 'nscount': 0, 'arcount': 0, 'qd': <DNSQR  qname='google.com.' qtype=TXT qclass=IN |>, 'an': None, 'ns': None, 'ar': None}
16
{'length': None, 'id': 4146, 'qr': 1, 'opcode': 0, 'aa': 0, 'tc': 0, 'rd': 1, 'ra': 1, 'z': 0, 'ad': 0, 'cd': 0, 'rcode': 0, 'qdcount': 1, 'ancount': 1, 'nscount': 0, 'arcount': 0, 'qd': <DNSQR  qname='google.com.' qtype=TXT qclass=IN |>, 'an': <DNSRR  rrname='google.com.' type=TXT rclass=IN ttl=270 rdlen=None rdata=[b'v=spf1 ptr ?all'] |>, 'ns': None, 'ar': None}
15
{'length': None, 'id': 63343, 'qr': 0, 'opcode': 0, 'aa': 0, 'tc': 0, 'rd': 1, 'ra': 0, 'z': 0, 'ad': 0, 'cd': 0, 'rcode': 0, 'qdcount': 1, 'ancount': 0, 'nscount': 0, 'arcount': 0, 'qd': <DNSQR  qname='google.com.' qtype=MX qclass=IN |>, 'an': None, 'ns': None, 'ar': None}
15
{'length': None, 'id': 63343, 'qr': 1, 'opcode': 0, 'aa': 0, 'tc': 0, '

In [3]:
dns_query_a = f[8]['DNS']
dns_response_a = f[9]['DNS']

In [36]:
print(dns_query_a.fields, getattr(dns_query_a, 'qd').fields)
print(dns_response_a.fields, getattr(dns_response_a, 'qd').fields)
print(getattr(dns_response_a, 'an').fields)

{'length': None, 'id': 30144, 'qr': 0, 'opcode': 0, 'aa': 0, 'tc': 0, 'rd': 1, 'ra': 0, 'z': 0, 'ad': 0, 'cd': 0, 'rcode': 0, 'qdcount': 1, 'ancount': 0, 'nscount': 0, 'arcount': 0, 'qd': <DNSQR  qname='www.netbsd.org.' qtype=A qclass=IN |>, 'an': None, 'ns': None, 'ar': None} {'qname': b'www.netbsd.org.', 'qtype': 1, 'qclass': 1}
{'length': None, 'id': 30144, 'qr': 1, 'opcode': 0, 'aa': 0, 'tc': 0, 'rd': 1, 'ra': 1, 'z': 0, 'ad': 0, 'cd': 0, 'rcode': 0, 'qdcount': 1, 'ancount': 1, 'nscount': 0, 'arcount': 0, 'qd': <DNSQR  qname='www.netbsd.org.' qtype=A qclass=IN |>, 'an': <DNSRR  rrname='www.netbsd.org.' type=A rclass=IN ttl=82159 rdlen=None rdata=204.152.190.12 |>, 'ns': None, 'ar': None} {'qname': b'www.netbsd.org.', 'qtype': 1, 'qclass': 1}
{'rrname': b'www.netbsd.org.', 'type': 1, 'rclass': 1, 'ttl': 82159, 'rdlen': None, 'rdata': '204.152.190.12'}


In [45]:
print('DNS Query')
for key in dns_query_a.fields.keys():
    print(f'{key}:{getattr(dns_query_a, key)}', end=', ')

DNS Query
length:None, id:30144, qr:0, opcode:0, aa:0, tc:0, rd:1, ra:0, z:0, ad:0, cd:0, rcode:0, qdcount:1, ancount:0, nscount:0, arcount:0, qd:b'\x03www\x06netbsd\x03org\x00\x00\x01\x00\x01', an:None, ns:None, ar:None, 

In [44]:
print('DNS Response')
for key in dns_response_a.fields.keys():
    print(f'{key}:{getattr(dns_response_a, key)}', end=', ')

DNS Response
length:None, id:30144, qr:1, opcode:0, aa:0, tc:0, rd:1, ra:1, z:0, ad:0, cd:0, rcode:0, qdcount:1, ancount:1, nscount:0, arcount:0, qd:b'\x03www\x06netbsd\x03org\x00\x00\x01\x00\x01', an:b'\x03www\x06netbsd\x03org\x00\x00\x01\x00\x01\x00\x01@\xef\x00\x04\xcc\x98\xbe\x0c', ns:None, ar:None, 

In [5]:
dns_query_a = getattr(dns_query_a, 'qd')

In [27]:
type(getattr(dns_query_a, 'qname'))
getattr(dns_query_a, 'qname').decode('utf-8')

'www.netbsd.org.'

In [30]:
for packet in f:
    if packet.getlayer(3) and packet.getlayer(3).name == 'DNS':
        dns_packet = packet.getlayer(3)
        if getattr(dns_packet, 'id') == 478:
            dns_an = getattr(dns_packet, 'an')
            if dns_an:
                print(dns_an.fields)
                print(dns_packet.fields)

{'rrname': b'ogs.google.com.', 'type': 5, 'rclass': 1, 'ttl': 3332, 'rdlen': None, 'rdata': b'www3.l.google.com.'}
{'length': None, 'id': 478, 'qr': 1, 'opcode': 0, 'aa': 0, 'tc': 0, 'rd': 1, 'ra': 1, 'z': 0, 'ad': 0, 'cd': 0, 'rcode': 0, 'qdcount': 1, 'ancount': 2, 'nscount': 0, 'arcount': 0, 'qd': <DNSQR  qname='ogs.google.com.' qtype=A qclass=IN |>, 'an': <DNSRR  rrname='ogs.google.com.' type=CNAME rclass=IN ttl=3332 rdlen=None rdata='www3.l.google.com.' |<DNSRR  rrname='www3.l.google.com.' type=A rclass=IN ttl=299 rdlen=None rdata=142.250.183.78 |>>, 'ns': None, 'ar': None}


In [5]:
for packet in f[:10]:
    print(packet.getlayer(0).fields)

{'dst': '3c:84:6a:e2:5e:46', 'src': '28:3a:4d:4a:a7:21', 'type': 2048}
{'dst': '3c:84:6a:e2:5e:46', 'src': '28:3a:4d:4a:a7:21', 'type': 2048}
{'dst': '28:3a:4d:4a:a7:21', 'src': '3c:84:6a:e2:5e:46', 'type': 2048}
{'dst': '28:3a:4d:4a:a7:21', 'src': '3c:84:6a:e2:5e:46', 'type': 2048}
{'dst': '3c:84:6a:e2:5e:46', 'src': '28:3a:4d:4a:a7:21', 'type': 2048}
{'dst': '28:3a:4d:4a:a7:21', 'src': '3c:84:6a:e2:5e:46', 'type': 2048}
{'dst': '3c:84:6a:e2:5e:46', 'src': '28:3a:4d:4a:a7:21', 'type': 2048}
{'dst': '3c:84:6a:e2:5e:46', 'src': '28:3a:4d:4a:a7:21', 'type': 2048}
{'dst': '28:3a:4d:4a:a7:21', 'src': '3c:84:6a:e2:5e:46', 'type': 2048}
{'dst': '28:3a:4d:4a:a7:21', 'src': '3c:84:6a:e2:5e:46', 'type': 2048}


## ICMP Stuff 

In [2]:
packets = rdpcap('../samples/icmp.pcap')

In [8]:
for packet in packets:
    if packet.getlayer(2).name == 'ICMP':
        icmp_packet = packet.getlayer(2)
        print(icmp_packet.fields)
        print(icmp_packet.time)
        print(getattr(icmp_packet, 'type') == 8)

{'type': 8, 'code': 0, 'chksum': 59492, 'id': 1, 'seq': 1, 'ts_ori': None, 'ts_rx': None, 'ts_tx': None, 'gw': None, 'ptr': None, 'reserved': None, 'length': None, 'addr_mask': None, 'nexthopmtu': None, 'unused': None}
1616591107.220834
True
{'type': 0, 'code': 0, 'chksum': 61540, 'id': 1, 'seq': 1, 'ts_ori': None, 'ts_rx': None, 'ts_tx': None, 'gw': None, 'ptr': None, 'reserved': None, 'length': None, 'addr_mask': None, 'nexthopmtu': None, 'unused': None}
1616591107.221228
False
{'type': 8, 'code': 0, 'chksum': 40286, 'id': 1, 'seq': 2, 'ts_ori': None, 'ts_rx': None, 'ts_tx': None, 'gw': None, 'ptr': None, 'reserved': None, 'length': None, 'addr_mask': None, 'nexthopmtu': None, 'unused': None}
1616591107.221512
True
{'type': 0, 'code': 0, 'chksum': 42334, 'id': 1, 'seq': 2, 'ts_ori': None, 'ts_rx': None, 'ts_tx': None, 'gw': None, 'ptr': None, 'reserved': None, 'length': None, 'addr_mask': None, 'nexthopmtu': None, 'unused': None}
1616591107.2217686
False
{'type': 8, 'code': 0, 'chksu

AttributeError: 'NoneType' object has no attribute 'name'

## UDP WebRTC Stuff

In [2]:
f = rdpcap('../samples/webrtc_teams.pcap')

In [10]:
for packet in f[349:600]:
    if packet.getlayer(2) and packet.getlayer(2).name == 'UDP':
        #print(packet['UDP'].fields)
        data = getattr(packet.getlayer(3), 'load')
        print(len(data))

136
136
148
100
136
216
64
160
160
100
216
216
216
216
216
116
148
100
100
216
216
216
116
160
100
216
216
216
216
216
216
132
20
132
232
216
216
144
216
216
100
132
216
216
216
32
216
216
100
100
216
116
216
216
216
216
216
216
216
100
216
116
216
216
216
216
216
216
216
216
232
216
176
656
216
216
216
216
216
216
216
44
184
184
184
184
184
184
184
184
184
184
1164
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
184
108
1164
184
68
100
184
184
184
184
184
184
184
184
184
184
184


## ARP Stuff

In [2]:
f = rdpcap('../samples/arp-storm.pcap')

In [7]:
for packet in f:
    print(packet.getlayer(1).name)

ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
ARP
